# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.lesoir.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"r-article--link"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

88

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.lesoir.be/478559/article/2022-11-22/sept-familles-sur-dix-ont-du-mal-concilier-vie-privee-et-vie-professionnelle',
  'Sept familles sur dix ont du mal à concilier vie privée et vie professionnelle'],
 ['https://www.lesoir.be/478630/article/2022-11-22/rd-congo-la-domination-des-colons-belges-merite-reparation',
  'RD Congo: la domination des colons belges mérite réparation'],
 ['https://www.lesoir.be/478631/article/2022-11-22/coupe-du-monde-vers-un-retour-de-romelu-lukaku-contre-le-maroc',
  'Coupe du monde: vers un retour de Romelu Lukaku contre le Maroc?'],
 ['https://www.lesoir.behttp://soirmag.lesoir.be/478610/article/2022-11-22/lex-premiere-ministre-danoise-brave-les-interdits-avec-une-tenue-symbolique-au',
  'L’ex-Première ministre danoise brave les interdits avec une tenue symbolique au Mondial (photos)'],
 ['https://www.lesoir.be/478627/article/2022-11-22/votre-epargne-devrait-bientot-etre-mieux-remuneree',
  'Votre épargne devrait bientôt être mieux rémunérée'],


## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.lesoir.be/478559/article/2022-11-2...,Sept familles sur dix ont du mal à concilier v...
1,https://www.lesoir.be/478630/article/2022-11-2...,RD Congo: la domination des colons belges méri...
2,https://www.lesoir.be/478631/article/2022-11-2...,Coupe du monde: vers un retour de Romelu Lukak...
3,https://www.lesoir.behttp://soirmag.lesoir.be/...,L’ex-Première ministre danoise brave les inter...
4,https://www.lesoir.be/478627/article/2022-11-2...,Votre épargne devrait bientôt être mieux rémun...
...,...,...
83,https://www.lesoir.be/476670/article/2022-11-1...,Envoyer un colis: une opération simplifiée à m...
84,https://www.lesoir.be/476456/article/2022-11-1...,"Les stations-services, nouveaux «lieux de soci..."
85,https://www.lesoir.be/476543/article/2022-11-1...,Etude: comment les enfants vivent la garde alt...
86,https://www.lesoir.be/476493/article/2022-11-1...,Les Belges face à la crise – La famille Monami...


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/lesoir_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')
chapo = soup.find("r-article--chapo").get_text(separator=' ')
content = soup.find("r-article--section").get_text(separator=' ')

print(title)
print("==================================")
print(chapo)
print("==================================")
print(content)

Sept familles sur dix ont du mal à concilier vie privée et vie professionnelle
Le Baromètre 2022 de la Ligue des familles relève que plus de 80 % des parents sont favorables au télétravail. Il constate aussi que les inégalités perdurent entre les femmes et les hommes.
  C ’est le défi majeur des parents ! Le nouveau Baromètre de la ligue des Familles pour 2022 nous apprend que 70 % d’entre eux estiment que la conciliation de leur vie professionnelle, à temps plein, et de leur vie privée, avec les enfants, reste très compliquée. Surtout pour les femmes. Un pourcentage qui grimpe à 81 % quand il s’agit des parents d’enfants de 0 à 3 ans. C’est que les horaires de travail sont en inadéquation avec les horaires des écoles et même des garderies scolaires. 



### Nettoyage du texte à l'aide d'expressions régulières

In [8]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


  C ’est le défi majeur des parents ! Le nouveau Baromètre de la ligue des Familles pour 2022 nous apprend que 70 % d’entre eux estiment que la conciliation de leur vie professionnelle, à temps plein, et de leur vie privée, avec les enfants, reste très compliquée. Surtout pour les femmes. Un pourcentage qui grimpe à 81 % quand il s’agit des parents d’enfants de 0 à 3 ans. C’est que les horaires de travail sont en inadéquation avec les horaires des écoles et même des garderies scolaires. 

 C ’est le défi majeur des parents ! Le nouveau Baromètre de la ligue des Familles pour 2022 nous apprend que 70 % d’entre eux estiment que la conciliation de leur vie professionnelle, à temps plein, et de leur vie privée, avec les enfants, reste très compliquée. Surtout pour les femmes. Un pourcentage qui grimpe à 81 % quand il s’agit des parents d’enfants de 0 à 3 ans. C’est que les horaires de travail sont en inadéquation avec les horaires des écoles et même des garderies scolaires. 


### Création d'un fichier avec le contenu de l'article


In [9]:
with open("../data/lesoir_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{chapo}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp